In [11]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark") \
    .getOrCreate()
#讀檔案
#dataRDD =spark.sparkContext.textFile("dataset/simpleText.txt")\
# .map(lambda line: line.split("\t"))\
# .map(lambda part: Row(label=float(part[0]), text=part[1]))
#dataDF = spark.createDataFrame(dataRDD)
data = spark.read.option("header","true")\
    .option("inferSchema", "true")\
    .option("delimiter", "|")\
    .csv("hotel_review_tokens_tw.txt")
# 訓練 測試 資料集
(train, test) = data.randomSplit([0.9,0.1 ], 1234)
#斷詞
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer,HashingTF, IDF
regexTokenizer = RegexTokenizer().\
    setInputCol("raw_words").\
    setOutputCol("words").\
    setPattern(" ")
#去除停用詞
#CountVectorizer詞頻
#最多幾個字詞? 取最高頻的
#統計字詞大於等於2次
tf = CountVectorizer().\
    setInputCol("words").\
    setOutputCol("tf_vector").\
    setVocabSize(30000).\
    setMinDF(1)
#HashingTF詞頻
hashingTF = HashingTF().\
    setInputCol("words").\
    setOutputCol("htf_vector").\
    setNumFeatures(10000)
idf = IDF().\
    setInputCol("htf_vector").\
    setOutputCol("htfidf_vector")

#搭配一個你想要的分類器MPL
from pyspark.ml.classification import MultilayerPerceptronClassifier
layers = [10000, 80, 40, 2]
trainer=MultilayerPerceptronClassifier()\
    .setLayers(layers)\
    .setLabelCol("label")\
    .setFeaturesCol("htfidf_vector")\
    .setPredictionCol("prediction")\
    .setBlockSize(128)\
    .setSeed(1234)\
    .setMaxIter(500)
# 這樣寫也可以
#trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers,
# labelCol="label", featuresCol="tfidf_vector" , blockSize=128, seed=1234)
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[regexTokenizer, tf, hashingTF, idf, trainer])
#pipeline = Pipeline().setStages([regexTokenizer, remover, tf, hashingTF, idf, trainer])
#進行訓練與測試
model = pipeline.fit(train)
result = model.transform(test)
#準確率
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label",
predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(result)
print("Test Accuracy = %.2f" % accuracy)
print("Test Error = %.2f" % (1.0 - accuracy))

Test Accuracy = 0.92
Test Error = 0.08


In [18]:
import jieba
import re
review = input("請輸入一段文字:")
jieba.set_dictionary('jieba/jieba_big_chinese_dict/dict.txt.big')

tokens = []
seg_list = jieba.lcut(str(review), cut_all=False)
tokens.append( " ".join(seg_list ))
#看切詞
tokens
#建立新的DataFrame
newdata = spark.createDataFrame( [tokens] ,['raw_words'])
#模型訓練
result = model.transform(newdata)
#模型預測
prob = result.select(['probability']).toPandas()['probability'].iloc[0].tolist()
print("正向機率:"+str(round(prob[1],2)))
print("負向機率:"+str(round(prob[0],2)))

請輸入一段文字:不愉快


Building prefix dict from C:\Users\Sam\Desktop\machine learning\jieba\jieba_big_chinese_dict\dict.txt.big ...
Loading model from cache C:\Users\Sam\AppData\Local\Temp\jieba.u22256f6af0add31db41b150801fcc58d.cache
Loading model cost 0.947 seconds.
Prefix dict has been built succesfully.


正向機率:0.0
負向機率:1.0
